<a href="https://colab.research.google.com/github/sonam-pankaj95/vectordb/blob/main/movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural search demo - initial indexing

Code in this notebook shows how to prepare data for indexing in a vector search engine.

It contains the following steps:

* Downloading text data which we want to search
* Initialization of pre-trained text vectorization models (with SentenceTransformer)
* Converting text data into vectors and saving it.

In [ ]:
# We use SentenceTransformer pre-trained models to convert our text into vectors.
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2e2d592085e463aecd4031001fbbe416a23f20753c8ac0a39faf77dd4a1f6d68
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
# This code will download and create a pre-trained sentence encoder

# all-MiniLM-L6-v2 - is a distilated (lightweight) version of MPNet model.
# It is optimized for the fast inference.
# Full list of available models could be found here https://www.sbert.net/docs/pretrained_models.html
model = SentenceTransformer('all-MiniLM-L6-v2', device="cuda")

In [ ]:
df = pd.read_json('./startups_demo.json', lines=True)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving movies_metadata.csv to movies_metadata.csv


In [ ]:
df = pd.read_csv('movies_metadata.csv')

<ipython-input-12-a87af5d0a418>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_metadata.csv')


In [ ]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [ ]:
df_f = pd.DataFrame(df,columns=['original_title','genres','vote_average'])

In [ ]:
df_f

,original_title,genres,vote_average
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",6.5
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",6.1
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",5.7
...,...,...,...
45461,رگ خواب,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",4.0
45462,Siglo ng Pagluluwal,"[{'id': 18, 'name': 'Drama'}]",9.0
45463,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",3.8
45464,Satana likuyushchiy,[],0.0


In [ ]:
(df_f['genres'][0])

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [ ]:

import ast


# Given string
input_string = df_f['genres'][0]

# Parse the string into a list of dictionaries
list_of_dictionaries = ast.literal_eval(input_string)

# Extract 'name' values from dictionaries
names = [item['name'] for item in list_of_dictionaries]

# Print the extracted names
print(names[:])


['Animation', 'Comedy', 'Family']


In [ ]:
# Here we encode all startup descriptions
# We do encoding in batches, as this reduces overhead costs and significantly speeds up the process
vectors = model.encode([
    row.original_title + ". " +   row.genres

    for row in df_f.itertuples()
], show_progress_bar=True)

Batches:   0%|          | 0/1421 [00:00<?, ?it/s]

In [ ]:
# Now we have all our descriptions converted into vectors.
# We have 40474 vectors of 384 dimentions. The output layer of the model has this dimension
vectors.shape

(45466, 384)

In [ ]:
# You can download this saved vectors and continue with rest part of the tutorial.
np.save('vectors.npy', vectors, allow_pickle=False)

In [ ]:
from google.colab import files
files.download('vectors.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Optional part - make a test query

Let's just make sure, that our vectors are correctly converted and make sense.

For this we manually search for a closest vectors of a random sample.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Take a random description as a query
sample_query = "Toy story"
print(sample_query)

Toy story


In [ ]:
query_vector = model.encode(sample_query)  # Convert query description into a vector.

In [ ]:
scores = cosine_similarity([query_vector], vectors)[0]  # Look for the most similar vectors, manually score all vectors
top_scores_ids = np.argsort(scores)[-5:][::-1]  # Select top-5 with vectors the largest scores

In [ ]:
# Check if result similar to the query
for top_id in top_scores_ids:
  print(df.iloc[top_id].original_title)
  print("-----")

Toy Story That Time Forgot
-----
The Toy
-----
Toy Story 2
-----
Toy Story 3
-----
Plastic Galaxy: The Story of Star Wars Toys
-----
